In [4]:
# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

In [1]:
from transformers import GPT2LMHeadModel
gpt2_hf = GPT2LMHeadModel.from_pretrained("gpt2") #initiating the init file of gpt2


In [4]:
sd_gpt2 = gpt2_hf.state_dict()
for k,v in sd_gpt2.items():
    print(k)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

In [5]:
sd_gpt2["transformer.wte.weight"].view(-1).max()

tensor(1.7852)

In [6]:
sd_gpt2["transformer.wte.weight"].view(-1)[:10]

tensor([-0.1101, -0.0393,  0.0331,  0.1338, -0.0485, -0.0789, -0.2398, -0.0895,
         0.0253, -0.1074])

In [5]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-21 13:27:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-09-21 13:27:56 (11.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [6]:
#read the file to inspect
with open(file='./data/input.txt', mode='r',  encoding='utf-8') as f:
    text = f.read()
    

In [7]:
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [8]:
f'len text:{len(text)}'
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'number of unique chars:{chars}')
print(vocab_size)


number of unique chars:['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [9]:
#tokenization 
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
#enocde
encode = lambda s: [stoi[c] for c in s] 
#decode
decode = lambda numz: ''.join([itos[i] for i in numz])

In [10]:
#encode('hi there!')
print(decode(encode('hi there!')))

hi there!


In [11]:
#encode entire data 
import torch
data = torch.tensor(encode(text), dtype= torch.long)

#same bc special case of having no dimension 
test = data.view(-1)[:1000] #self.long() is equivalent to self.to(torch.int64).
print(test.shape)

print(data.shape, data.dtype)
print(data[:100])


torch.Size([1000])
torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
#split to train/test
n = int(0.9 * len(data))
train_set = data[:n]
test_set = data[n:]

In [13]:
block_size = 8
train_set[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
#data loader 
#time dimension
x = train_set[:block_size]
y = train_set[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1] #to include t 
    target= y[t]
    print(f'when context is {context} we predict {target}')



when context is tensor([18]) we predict 47
when context is tensor([18, 47]) we predict 56
when context is tensor([18, 47, 56]) we predict 57
when context is tensor([18, 47, 56, 57]) we predict 58
when context is tensor([18, 47, 56, 57, 58]) we predict 1
when context is tensor([18, 47, 56, 57, 58,  1]) we predict 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]) we predict 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) we predict 58


In [15]:
#make use of prallalism; So a couple of same-size blocks 
#bTCH Dimension
torch.manual_seed(1337)
batch_size = 4  # independent sequences to be processed in parallel every forward/backward 
block_size = 8 #maximum context len in each of the sequence, can be less than block size! in each of the predictions

def get_batch(split):
    data = train_set if split == "train" else test_set
    
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1: i + block_size+1] for i in ix])
    return x, y

x_batch, y_b = get_batch(split='train')
print(f'input batch size is {x_batch.shape}')
print(f'the x_batch is: \n {x_batch}')

print('=='*20)

print(f'target y_batch size is {y_b.shape}')
print(f'the x_batch is: \n {y_b}')

print('=='*20)

for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = x_batch[b, :t+1]
        target = y_b[b, t]
        print(f'when the context is {context.tolist()} we are predicting {target}')

# 32 independent exp packed in a single batch 


input batch size is torch.Size([4, 8])
the x_batch is: 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target y_batch size is torch.Size([4, 8])
the x_batch is: 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when the context is [24] we are predicting 43
when the context is [24, 43] we are predicting 58
when the context is [24, 43, 58] we are predicting 5
when the context is [24, 43, 58, 5] we are predicting 57
when the context is [24, 43, 58, 5, 57] we are predicting 1
when the context is [24, 43, 58, 5, 57, 1] we are predicting 46
when the context is [24, 43, 58, 5, 57, 1, 46] we are predicting 43
when the context is [24, 43, 58, 5, 57, 1, 46, 43] we are predicting 39
when the context is [44] we are predicting 53
when the context is [44, 53] 

In [16]:
print(x_batch) #input to the transformer 

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [17]:
#Baseline: bigram 
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        """ nn.Embedding is a wrapper
        input: list of indices.
        out: corresponding word embeddings
        """
        super().__init__()
        #wrapper 
        self.token_embeding_table = nn.Embedding(vocab_size, vocab_size) 
        
    def forward(self, idx, target=None):
        logits = self.token_embeding_table(idx) #(B,T, C)
        #print(logits.shape)
        if target is None:
            loss = None 
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #streched 
            # print(f'logits shape is {logits.shape} and it looks like:')
            # print(logits)
            target = target.view(B*T) #-1 streched 
            # print(f'logits shape is {target.shape} and it looks like:')
            # print(target)
            loss = F.cross_entropy(logits, target) #should be (B*T,C)  # LOSS: -ln(1/65)
            #print(f'idx shape is {idx.shape}')
        return logits, loss
    
    
    def generate(self, idx, max_new_tokens):
        #idx shape is (B, T) which is the current context within some batch
        #so the job of generate is to take the (B,T) and extend it to (B, T + 1) in all the time dimensions within the batches
        for _ in range(max_new_tokens):
            #get predictions
            logits, loss = self(idx)
            logits = logits[:,-1,:] #becomes (B, C)
            
            probs = F.softmax(logits, dim=-1) #(B, C)
            #sample from this probs
            # If input is a matrix with m rows, out is an matrix of shape (m * num_samples)
            idx_next = torch.multinomial(probs, num_samples=1) # (B * 1) 
            #appending the sample to the current seq
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 

In [18]:
#inspect forward pass     
obj = BigramLanguageModel(vocab_size)
logits, loss = obj(x_batch, y_b)
print(logits.shape)
print(loss) 
# idx is (B,T) , generate works on B then we gotta unplog the created one [0]
# zeros was \n 
print(decode(obj.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [19]:
#inspect nn.Embedding
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 5]])
embed = nn.Embedding(6, 2)
print(embed(input))

tensor([[[ 1.4021,  0.7641],
         [-1.0255, -1.2937],
         [ 0.6774, -1.0308],
         [-0.4239,  0.6698]],

        [[ 0.6774, -1.0308],
         [ 0.3770,  2.2209],
         [-1.0255, -1.2937],
         [-0.4239,  0.6698]]], grad_fn=<EmbeddingBackward0>)


In [20]:
#lets now train the model 
optim = torch.optim.AdamW(obj.parameters(),  lr=1e-3)

In [22]:
batch_size = 32
for steps in range(1000):
    # get a sample batch 
    x_batch, y_b = get_batch(split='train')
    
    #forward/backward
    # evaluate the loss
    logits, loss = obj( x_batch, y_b)
    #zero out all the gradients from prev steps 
    optim.zero_grad(set_to_none=True)
    #gettting the gradients for all the params 
    loss.backward()
    #using those gradients to update our params
    optim.step()


print(f' in step {steps} loss is: {loss.item()}')

    

 in step 999 loss is: 2.4415125846862793


In [23]:
print(decode(obj.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


US lawevengoft tho mooreawe he m!

This ghespins mainourerkenowoverch t HAne inong. wighe?
I:
RDUg, 


In [24]:
print(decode(obj.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


att me I y?
Footo! has wh-OMar be hots. VEObllan
Plouls plesthen ther a be wean I pr itrg t per mat.
y to f iserd me l ms moe!
BO,
ELe.
LI thir:
Watheas n:
MI's'd, V:

I.
T:
Thfer, t nt hore m I rathoue hethy,-tshy ly.
YBE:
Hamathe doulond wha my
Y:
De.
Cor y my?
SCKERUThy&xqupacoor l it igr mes ownd y?-esageqThatou bethour t

Aprisl bicther d I borchiordugovand nepellithipamer warmerur and mprothy w! Lirilmowobegrileaviere fougr nethealvith a gouromat my y panghesthe?
ueaye haly tokersos ck. bu


In [25]:
#Attention inspection
#toy exp
import torch 

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape


torch.Size([4, 8, 2])

In [26]:
#version1: Average type interacrion of the tokens up to current token as the simpleset possible interaction way
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #including current token (t,C)
        xbow[b,t] = torch.mean(xprev, 0) #over zero dim which id time 


In [27]:
#version 2; using matrix mul
# back up for inspecting the matrix mul
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1 ,keepdim=True)
xbow2 = wei @ x #(T,T) * (B, T, C) ==> (B, T, C) broadcasting in pytorch the T,T will be B,T,T then for every batch 
# it happens on the level of batch, matrix mul on each of the batches in parallel and indivisually 
torch.allclose(xbow[0], xbow2[0])

True

In [28]:
#version 3: using softmx
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = tril.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wbow3 = wei @ x
torch.allclose(xbow[0], wbow3[0])

#TAKEOUT: we can do weighted aggregation of the past elements by Matrix Mul of the lower tril and the elements in this lower tril tells us how much this element is fuses in this positon!


True

In [29]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [30]:

torch.manual_seed(42)

a = torch.tril(torch.ones(3,3)) 
a = a/ torch.sum(a, 1, keepdim=True)    #normalizing a: all rows sum to 1 ~ softmax 
b = torch.randint(0,10, (3,2)).float()
c = a @ b # (3*3) (3,2) => 3*2


print(f'a=')
print(a)
print(f'b=')
print(b)
print(f'c = a @ b')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c = a @ b
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [31]:
#inspect difference nn.Linear vs nn.Embedding

In [32]:
a = nn.Linear(10, 2)
a 

Linear(in_features=10, out_features=2, bias=True)

## Note on difference nn.Linear and nn.Embedding 

An Embedding layer is essentially just a Linear layer. So you could define a your layer as nn.Linear(1000, 30), and represent each word as a one-hot vector, e.g., [0,0,1,0,...,0] (the length of the vector is 1,000).

As you can see, any word is a unique vector of size 1,000 with a 1 in a unique position, compared to all other words. Now giving such a vector v with v[2]=1 (cf. example vector above) to the Linear layer gives you simply the 2nd row of that layer.

nn.Embedding just simplifies this. Instead of giving it a big one-hot vector, you just give it an index. This index basically is the same as the position of the single 1 in the one-hot vector.



‘nn.Embedding’ is no architecture, it’s a simple layer at best. In fact, it’s a linear layer just with a specific use.

Internally, nn.Embedding is – like a linear layer – a M x N matrix, with M being the number of words and N being the size of each word vector. There’s nothing more to it. It just matches a word (specified by an index) to the corresponding word vector, i.e., the corresponding row in the matrix.

In [33]:
#inspect on nn.Linear
m = nn.Linear(16, 16)
input = torch.randn(2, 3, 16)
output = m(input)
print(output.size())

torch.Size([2, 3, 16])


In [34]:
output

tensor([[[ 1.2935e-01, -5.4255e-02,  8.6804e-01,  7.1032e-01,  6.4082e-01,
           1.2002e+00,  7.3290e-01,  9.9904e-01,  3.0008e-01, -2.6067e-01,
          -1.5186e-02, -3.1486e-01, -1.2295e+00,  4.9245e-02,  1.2247e-02,
           8.3614e-01],
         [-1.0657e+00, -1.0875e-01, -2.2117e-02,  1.2596e+00,  1.0819e+00,
          -5.9091e-01, -1.0574e+00, -1.2076e+00,  5.5400e-01,  2.9481e-01,
          -6.0595e-01, -1.8302e+00, -1.1210e+00, -8.2700e-01, -2.6840e-01,
          -7.9106e-01],
         [ 2.4123e-01, -1.8315e-01, -5.6941e-02,  5.0335e-01, -3.5398e-01,
          -5.4684e-01, -3.5876e-01, -5.8759e-01, -4.7856e-01,  2.5169e-02,
           1.4939e-01, -4.3014e-01, -1.4933e-01, -3.4374e-01,  7.2644e-01,
           2.4808e-02]],

        [[-2.8538e-01,  6.1068e-01,  4.4531e-01,  6.3977e-01, -3.6194e-01,
          -9.6942e-02, -1.1857e+00, -6.5978e-01, -8.1429e-01,  7.4203e-01,
          -6.2159e-01, -4.8497e-01, -1.4999e-01, -3.4138e-01, -4.7673e-04,
          -9.8899e-02],
  

In [73]:
#version 4: Self-aatention: differece w/ version 3, is the weights instead of uniformely distributed, should be data dependent
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.rand(B,T,C)

head_size = 16
#each token in the B,T fashion will emit 2 vectors 
key = nn.Linear(C, head_size, bias=False) #apply matrix mul
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B, T, head_size
q = query(x) #  B, T, head_size
v = value(x)
wei = q @ k.transpose(-2,-1) # (B, T, head_size) * (B, head_size, T) ==> (B,T,T)

tril = torch.tril(torch.ones(T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

#out = wei # x
out = wei @ v # (B, T, head_size) 4,8,16
out.size()

torch.Size([4, 8, 16])

In [72]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4409, 0.5591, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2975, 0.3373, 0.3652, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2211, 0.2898, 0.2236, 0.2654, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1832, 0.2163, 0.1954, 0.2437, 0.1614, 0.0000, 0.0000, 0.0000],
        [0.1330, 0.2227, 0.1784, 0.2159, 0.1044, 0.1456, 0.0000, 0.0000],
        [0.1283, 0.1367, 0.1385, 0.1522, 0.1083, 0.1341, 0.2021, 0.0000],
        [0.1064, 0.1332, 0.1265, 0.1445, 0.0940, 0.1200, 0.1231, 0.1524]],
       grad_fn=<SelectBackward0>)

In [67]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

tensor([1., 0., 0., 0., 0., 0., 0., 0.])

In [74]:
#INSPECT On sqrt head_size ** 0.5
a = torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])